In [1]:
import scipy.io as sio
import numpy as np
import tkinter as tk
from tkinter.filedialog import askdirectory
from Bioacustica_Com1 import Metodologia,ZscoreMV, lamda_unsup, segmentacion, seleccion_features, time_and_date

In [2]:
def mlamda_fuzzy_3pi_apract(it_num,dat_norma,mean_class,flag):
    #data = sio.loadmat('mlamda.mat')

    #dat_norma = data['dat_norma1']
    #mean_class=data['mean_class']

    #[clusters,xx]=size(mean_class);
    clusters,xx=np.shape(mean_class)
    #[num_clases,xx]=size(mean_class);
    num_clases,xx = np.shape(mean_class)
    #[num_datos, num_feat] = size(dat_norma1);
    num_datos, num_feat = np.shape(dat_norma)
    #mean_class = [ones(1,num_feat)*0.5; mean_class];
    mean_class=np.append((np.ones((1,num_feat), np.double))*0.5,mean_class,axis=0)
    #conteo = [zeros(1,1000)]; %hasta 1000 clases
    conteo = np.zeros((1,1000),np.double )  #hasta 1000 clases
    #winsorizacion prueba
    #[n,m]=size(dat_norma1);
    n,m=np.shape(dat_norma)

    if flag==1:
        # for t =1:it_num
        for t in range(1, it_num+1):
           
            # for j = 1:num_datos
            for j in range(0, num_datos):
               
                #mads = (mean_class.^(dat_norma(j*ones(size(mean_class,1),1),:))).*((1-mean_class).^(1-dat_norma(j*ones(size(mean_class,1),1),:)));
                mads=(mean_class**dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:])*((1-mean_class)**(1-dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:]))
                # gads = prod(mads,2)./(prod(mads,2)+(prod(1-mads,2)));
                gads = np.prod(mads,axis=1)/(np.prod(mads,axis=1)+(np.prod(1-mads,axis=1)))        
                #[x,i_gad]=max(gads);
                x,i_gad=(np.max(gads)), np.argmax(gads)

                if i_gad == 0:
                    num_clases = num_clases + 1
                    # conteo(num_clases)=1;
                    conteo[0,num_clases-1]=1
                    # conteo = np.append(conteo, 1)
                    # mean_clas = [mean_clas;mean([mean_clas(1,:);dat_norma(j,:)])];
                    mean_class = np.append(mean_class, np.expand_dims((np.mean(([mean_class[0, :], dat_norma[j, :]]), axis=0)), axis=0), axis=0)
                   
                else:      
                    #conteo(i_gad)=conteo(i_gad)+1;
                    conteo[0,i_gad]=conteo[0,i_gad]+1
                    #new_mean = mean_clas(i_gad,:) + (1/(conteo(i_gad)))*(dat_norma(j,:)-mean_clas(i_gad,:))
                    new_mean = np.expand_dims(mean_class[i_gad,:] + (1/(conteo[0,i_gad]))*(dat_norma[j,:]-mean_class[i_gad,:]),axis=0)
                    #mean_clas(i_gad,:) = new_mean;
                    mean_class[i_gad,:] = new_mean  
       
        #gadso = zeros(num_clases,num_datos);%num_clases para retirar la NIC
        gadso = np.zeros([num_clases,num_datos])
        #starts the assignment of data to the classes
        for j in range(0,num_datos):
            # mads = (mean_class(2:end,:).^(dat_norma(j*ones(size(mean_class(2:end,:),1),1),:))).*((1-mean_class(2:end,:)).^(1-dat_norma(j*ones(size(mean_class(2:end,:),1),1),:)));  
            mads=(mean_class[1:,:]**dat_norma[(j)*np.ones(np.size(mean_class[1:,:],0),dtype=int) ,:])*((1-mean_class[1:,:])**(1-dat_norma[(j)*np.ones(np.size(mean_class[1:,:],0),dtype=int) ,:]))
            #gadso(:,j) = prod(mads,2)./(prod(mads,2)+(prod(1-mads,2)));
            gadso[:,j] = np.prod(mads,axis=1)/(np.prod(mads,axis=1)+(np.prod(1-mads,axis=1)))  
        #delete(Bar)
        #[x, recon] = max(gadso,[],1); %assign the data to the classes according to their maximum GAD
        x, recon = np.expand_dims((np.max(gadso,axis=0)),axis=0), np.expand_dims(np.argmax(gadso,axis=0),axis=0)
           
    else:
       
        #gadso = zeros(num_clases+1,num_datos); %num_clases para retirar la NIC
        gadso = np.zeros([num_clases+1,num_datos])

        #for j = 1:num_datos
        for j in range(0, num_datos):
            #mads = (mean_clas.^(mjota)).*((1-mean_clas).^(1-mjota)) de matlad
            #mads = (mean_clas**(mjota))*((1-mean_clas)**(1-mjota)) # python
            # mads = (mean_class.^(dat_norma(j*ones(size(mean_class,1),1),:))).*((1-mean_class).^(1-dat_norma(j*ones(size(mean_class,1),1),:)));
            mads=(mean_class**dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:])*((1-mean_class)**(1-dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:]))
            #gadso(:,j) = prod(mads,2)./(prod(mads,2)+(prod(1-mads,2)));
            gadso[:,j] = np.prod(mads, axis=1)/(np.prod(mads, axis=1)+np.prod(1-mads, axis=1))
        # [x, recon] = max(gadso,[],1);
        x, recon = np.expand_dims((np.max(gadso,axis=0)),axis=0), np.expand_dims(np.argmax(gadso,axis=0),axis=0)
    return gadso,recon,mean_class


# comparar datos entre Matlab y python con flag=1
 #IF=sio.loadmat('mlamda2.mat')
 #Gadso=IF['gadso']
#Mean_clas=IF['mean_clas']
#errMean_clas = mean_squared_error(Mean_clas, mean_class)
#errGadso = mean_squared_error(Gadso, gadso)  

# comparar datos entre Matlab y python con flag=0
 #IF=sio.loadmat('mlamdaNoFlag.mat')
 #Gadso=IF['gadso']
#Mean_clas=IF['mean_clas']    
#errMean_clas = mean_squared_error(Mean_clas, mean_class
#errGadso = mean_squared_error(Gadso, gadso)

In [3]:

specs=sio.loadmat("D:\Proyectos Visual Studio\Python-Jupyther\Bioacustica\Variables\Species.mat")
specs=np.array(specs["species"])
speciesStr=specs[0:,0]
speciesStr=np.array([i[0] for i in speciesStr])

In [4]:
indnormal = []
indmerged = []
spec2SchInd = []
for i in range(len(speciesStr)):
    for g in range(specs.shape[0]):
        if speciesStr[i] == specs[g, 0]:
            if len(specs[g, 0]) >= 7:
                if specs[g, 0][0:7] == 'Merged_':
                    indmerged.append(g)
                else:
                    indnormal.append(g)
            else:
                indnormal.append(g)
            spec2SchInd.append(g)


indmezclas = []
mean_class = []
mediafrecuencia =[]
stdfrecuencia = []

for k in spec2SchInd:
    if k in indmerged: #crep que nunca entra en este if, almenos en esta muestra
        params=specs[k][-1]
        for i in params:
            mean_class.append(i[0,:])
            mediafrecuencia.append(i[1,:])
            stdfrecuencia.append(i[2,:])
        indmezclas.append(specs[k][1].shape[0])
    else:
        params=specs[k][-1]
        mean_class.append(params[0,:])
        mediafrecuencia.append(params[1,:])
        stdfrecuencia.append(params[2,:])
        indmezclas.append(1)
mean_class=np.array(mean_class)
#e=mean_class
feat=np.concatenate(specs[indnormal,3])

for j in indmerged:
    feat=np.concatenate(feat,specs[indmerged,3])

feat=np.unique(feat)

a=np.concatenate(specs[indnormal,4])

for j in indmerged:
    a=np.concatenate(a,specs[indmerged,4])

b=np.concatenate(specs[indnormal,5])

for j in indmerged:
    b=np.concatenate(b,specs[indmerged,5])

mediafrecuencia=np.array(mediafrecuencia)
stdfrecuencia=np.array(stdfrecuencia)

In [12]:
datos=sio.loadmat("D:\Proyectos Visual Studio\Python-Jupyther\Bioacustica\Variables\datos_seg.mat")
datos=np.array(datos["datos"])
fs=44100

In [5]:
ruta='D:/Users/ACER/Desktop/Trabajo Investigacion/Aureas Mono especies/Audio1'
#ruta = askdirectory(title = 'Seleccionar carpeta con audios')
canal=1
banda=["min","max"]
if type(banda[0])==str and type(banda[1])==str:
    datos,nombre_archivo,fs=segmentacion(ruta,["min","max"],canal)
else:
    datos,nombre_archivo,fs=segmentacion(ruta,banda,canal)


In [12]:
try:
    datos_carac = datos[:, 12:] #era desde 12 pero en los cambios de abril lo ponen en 13
except:
    print("No se encontraron vocalizaciones")
    table = []
    Fecha = []
    recon = []


datos_carac1=datos[:,7:10]
#datos_julian=datos[:,6:10] #De la actualizacion del codigo
datos_carac2=[datos_carac1,datos_carac]
datos_carac = np.concatenate((datos_carac2), axis=1)

blocal=np.amin(datos_carac,axis=0)
alocal=np.amax(datos_carac,axis=0)

aglobal = np.vstack((a, alocal))
bglobal = np.vstack((b, blocal))
aglobal=np.amax(aglobal,axis=0)
bglobal=np.amin(bglobal,axis=0)
aglobal=np.reshape(aglobal,(1,len(aglobal)))
bglobal=np.reshape(bglobal,(1,len(bglobal)))

datosnorm=ZscoreMV (datos_carac,bglobal,aglobal-bglobal)
#aaa=datosnorm
a1 = np.where(datosnorm > 1)
b1 = np.where(datosnorm < 0)

datosnorm[a1]=1
datosnorm[b1]=0

datos_clasifi=datosnorm

infoZC=np.array([aglobal[0],bglobal[0],0],dtype=object)

for i in range(0,mean_class.shape[0]):
    mean_class[i, :] = (mean_class[i, :] * (a[i, :] - b[i, :]) + b[i, :] - bglobal) / (aglobal - bglobal)

feat=np.array(range(0,len(datos_clasifi[1])))
infoZC[2]=feat
datos_clasifi=datos_clasifi[:,:len(datos_clasifi[1])]

mean_clas2=[]

In [13]:
gadso,recon,mean_clas2 = mlamda_fuzzy_3pi_apract(10, datos_clasifi,mean_class,0)

meandom = mediafrecuencia[:, 0]
stddom = stdfrecuencia[:, 0] 
datosdom = datos_carac[:, 0].T
meandomt=np.resize(meandom,[1,meandom.shape[0]])
stddomt=np.resize(stddom,[1,stddom.shape[0]])
add=np.tile(datosdom, (meandom.shape[0], 1))
bmd=np.tile(meandomt.T, (1, datosdom.shape[0]))
csd=np.tile(stddomt.T, (1, datosdom.shape[0]))
memberdom=np.exp(-(add-bmd)**2/(2*csd**2))

Gadsdom=gadso[1:,:]
threshold = np.exp((-(2*stddom)**2)/(2*stddom**2))
thresholdt=np.resize(threshold,[1,stddom.shape[0]])
Gadsdom[memberdom < np.tile(thresholdt.T, (1, memberdom.shape[1]))] = 0
Gadsdom= np.vstack((gadso[0,:],Gadsdom))

#Sort=np.sort(Gadsdom,axis=0)[::-1]
Indices = np.argsort(Gadsdom,axis=0)[::-1]
recon = Indices[0,:]

In [8]:
repre = np.zeros(mean_clas2.shape[0], dtype=int)
for i in range(mean_clas2.shape[0]):
    Euc = np.zeros(datos_clasifi.shape[0])
    for j in range(datos_clasifi.shape[0]):
        V = mean_clas2[i, :] - datos_clasifi[j, :]
        Euc[j] = np.dot(V, V.T)
    repre[i] = np.argmin(Euc)

mediafrecuencia2=[]
stdfrecuencia2=[]

for i in range(0,mean_clas2.shape[0]):
    indclass2=np.where(recon==i)[0]
    mediafrecuencia2.append(np.mean(datos_carac[indclass2],axis=0))
    stdfrecuencia2.append(np.std(datos_carac[indclass2],axis=0))
mediafrecuencia2=np.array(mediafrecuencia2)
stdfrecuencia2=np.array(stdfrecuencia2)

frecuencia=np.array([mediafrecuencia2,stdfrecuencia2])
acc=0
aux_gadso=gadso[0,:]
aux_mean_clas=mean_clas2[0,:]



d:\Python\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\Python\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
d:\Python\lib\site-packages\numpy\core\_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
d:\Python\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
d:\Python\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [11]:

for i in range(len(indmerged)):
    mask = np.logical_and(indmerged[i] + acc >= recon, recon > acc)
    recon[np.where(mask)] = i + 1
    
    aux_gadso[i + 1, :] = np.max(gadso[acc + 1:indmerged[i] + acc + 1, :], axis=0)
    aux_mean_clas[i + 1, :] = np.mean(mean_clas2[acc + 1:indmerged[i] + acc + 1, :], axis=0)
    
    acc = indmerged[i] + acc

gadso =  aux_gadso
mean_clas2 = aux_mean_clas
salida=np.concatenate((datos[:,0:10],datos[:,10:12]*(fs/2)),axis=1)
recon=np.reshape(recon,(len(recon),1))
tarr=np.concatenate((salida,recon),axis=1)
table=np.concatenate((nombre_archivo,tarr),axis=1)



In [42]:
recon2=np.reshape(recon,(len(recon),1))

In [19]:
#(np.tile(datosdom, (meandom.shape[0], 1)) - np.tile(meandom, (1, datosdom.shape[0])))**2

indclass2=np.where(recon==0)
wf=datos_carac[indclass2]
np.std(wf,axis=0)

array([3.07961353e+03, 2.91516981e+03, 2.84884654e+03, 9.79666438e+00,
       2.07769823e+00, 1.16959147e+00, 9.63939004e-01, 1.01583400e+01,
       1.56560437e+00, 1.33921215e+00, 7.64281158e-01, 7.50801059e+00,
       2.12870640e+00, 1.06459957e+00, 9.71100485e-01, 6.51121633e+00,
       2.23760913e+00, 9.49000957e-01, 1.05239887e+00, 1.58865127e+00,
       6.90720185e-01, 3.63852836e-01, 3.82619420e-01, 2.15065312e+00,
       1.03906481e+00, 8.69193891e-01, 6.17179005e-01])

In [27]:

recon

array([[ 0,  9,  6,  6,  0,  2,  2,  6,  9, 10, 10,  9,  9,  9,  0,  9,
         9, 10,  5,  9, 10,  9,  9,  9,  2,  0, 10,  5,  0,  9,  9,  5,
         9,  0,  9, 10,  9,  9,  9,  9,  9,  5,  9,  2,  2,  0,  9,  9,
         2, 10,  9,  5,  9,  9,  0,  6,  9,  9,  9,  0,  9,  9,  0,  9,
         9,  9,  9,  9,  9,  9,  0,  0,  9,  9,  9,  6,  9,  9,  9,  0,
         2,  9,  9,  9,  9,  0,  9,  9,  4,  9,  5,  9,  9,  0,  9,  9,
         5,  9,  9,  9,  9,  9,  5,  0,  9,  0,  9,  2,  9,  9,  9,  9,
         0,  5,  9,  9,  9,  5,  0,  9,  1,  2,  2,  2,  2,  2,  0,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  4,  2,  2,  6,  2,  6,  2,  2,  2,  2,  2,  6,  2,
         2,  2,  2,  2,  0,  6,  2,  2,  0,  0,  2,  2,  2,  2,  5,  0,
         0,  9,  2,  9,  0,  0,  2,  2,  5,  2,  0,  5,  2,  9,  2,  2,
         2,  2,  0,  9,  0,  9,  1,  9,  9,  9,  2,  0,  9,  9,  2, 10,
         0,  2,  9,  0,  9, 10,  0,  9,  9,  0,  0,  0,  9,  5, 

In [ ]:
dat_norma=datos_clasifi
it_num=10
num_datos, num_feat = np.shape(dat_norma)
for t in range(1, it_num+1):
           
    # for j = 1:num_datos
    for j in range(0, num_datos):
        
        #mads = (mean_class.^(dat_norma(j*ones(size(mean_class,1),1),:))).*((1-mean_class).^(1-dat_norma(j*ones(size(mean_class,1),1),:)));
        mads=(mean_class**dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:])*((1-mean_class)**(1-dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:]))
        print((1-dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:]))
        # gads = prod(mads,2)./(prod(mads,2)+(prod(1-mads,2)));
        gads = np.prod(mads,axis=1)/(np.prod(mads,axis=1)+(np.prod(1-mads,axis=1)))   


In [ ]:

aglobal[0]

In [ ]:
data = sio.loadmat('mlamda.mat')

dat_norma = data['dat_norma1']
mean_class=data['mean_class']

it_num=10
flag=0
#[clusters,xx]=size(mean_class);
clusters,xx=np.shape(mean_class)
#[num_clases,xx]=size(mean_class);
num_clases,xx = np.shape(mean_class)
#[num_datos, num_feat] = size(dat_norma1);
num_datos, num_feat = np.shape(dat_norma)
#mean_class = [ones(1,num_feat)*0.5; mean_class];
mean_class=np.append((np.ones((1,num_feat), np.double))*0.5,mean_class,axis=0)
#conteo = [zeros(1,1000)]; %hasta 1000 clases
conteo = np.zeros((1,1000),np.double )  #hasta 1000 clases
#winsorizacion prueba
#[n,m]=size(dat_norma1);
n,m=np.shape(dat_norma)

if flag==1:
    # for t =1:it_num
    for t in range(1, it_num+1):
        
        # for j = 1:num_datos
        for j in range(0, num_datos):
            
            #mads = (mean_class.^(dat_norma(j*ones(size(mean_class,1),1),:))).*((1-mean_class).^(1-dat_norma(j*ones(size(mean_class,1),1),:)));
            mads=(mean_class**dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:])*((1-mean_class)**(1-dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:]))
            # gads = prod(mads,2)./(prod(mads,2)+(prod(1-mads,2)));
            gads = np.prod(mads,axis=1)/(np.prod(mads,axis=1)+(np.prod(1-mads,axis=1)))        
            #[x,i_gad]=max(gads);
            x,i_gad=(np.max(gads)), np.argmax(gads)

            if i_gad == 0:
                num_clases = num_clases + 1
                # conteo(num_clases)=1;
                conteo[0,num_clases-1]=1
                # conteo = np.append(conteo, 1)
                # mean_clas = [mean_clas;mean([mean_clas(1,:);dat_norma(j,:)])];
                mean_class = np.append(mean_class, np.expand_dims((np.mean(([mean_class[0, :], dat_norma[j, :]]), axis=0)), axis=0), axis=0)
                
            else:      
                #conteo(i_gad)=conteo(i_gad)+1;
                conteo[0,i_gad]=conteo[0,i_gad]+1
                #new_mean = mean_clas(i_gad,:) + (1/(conteo(i_gad)))*(dat_norma(j,:)-mean_clas(i_gad,:))
                new_mean = np.expand_dims(mean_class[i_gad,:] + (1/(conteo[0,i_gad]))*(dat_norma[j,:]-mean_class[i_gad,:]),axis=0)
                #mean_clas(i_gad,:) = new_mean;
                mean_class[i_gad,:] = new_mean  
    
    #gadso = zeros(num_clases,num_datos);%num_clases para retirar la NIC
    gadso = np.zeros([num_clases,num_datos])
    #starts the assignment of data to the classes
    for j in range(0,num_datos):
        # mads = (mean_class(2:end,:).^(dat_norma(j*ones(size(mean_class(2:end,:),1),1),:))).*((1-mean_class(2:end,:)).^(1-dat_norma(j*ones(size(mean_class(2:end,:),1),1),:)));  
        mads=(mean_class[1:,:]**dat_norma[(j)*np.ones(np.size(mean_class[1:,:],0),dtype=int) ,:])*((1-mean_class[1:,:])**(1-dat_norma[(j)*np.ones(np.size(mean_class[1:,:],0),dtype=int) ,:]))
        #gadso(:,j) = prod(mads,2)./(prod(mads,2)+(prod(1-mads,2)));
        gadso[:,j] = np.prod(mads,axis=1)/(np.prod(mads,axis=1)+(np.prod(1-mads,axis=1)))  
    #delete(Bar)
    #[x, recon] = max(gadso,[],1); %assign the data to the classes according to their maximum GAD
    x, recon = np.expand_dims((np.max(gadso,axis=0)),axis=0), np.expand_dims(np.argmax(gadso,axis=0),axis=0)
        
else:
    
    #gadso = zeros(num_clases+1,num_datos); %num_clases para retirar la NIC
    gadso = np.zeros([num_clases+1,num_datos])

    #for j = 1:num_datos
    for j in range(0, num_datos):
        #mads = (mean_clas.^(mjota)).*((1-mean_clas).^(1-mjota)) de matlad
        #mads = (mean_clas**(mjota))*((1-mean_clas)**(1-mjota)) # python
        # mads = (mean_class.^(dat_norma(j*ones(size(mean_class,1),1),:))).*((1-mean_class).^(1-dat_norma(j*ones(size(mean_class,1),1),:)));
        mads = (mean_class ** (dat_norma[j * np.ones((mean_class.shape[0], 1)), :])) * ((1 - mean_class) ** (1 - dat_norma[j * np.ones((mean_class.shape[0], 1)), :]))
        
        #gadso(:,j) = prod(mads,2)./(prod(mads,2)+(prod(1-mads,2)));
        gadso[:,j] = np.prod(mads, axis=1)/(np.prod(mads, axis=1)+np.prod(1-mads, axis=1))
    # [x, recon] = max(gadso,[],1);
    x, recon = np.expand_dims((np.max(gadso,axis=0)),axis=0), np.expand_dims(np.argmax(gadso,axis=0),axis=0)

In [ ]:
mads=(mean_class**dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:])*((1-mean_class)**(1-dat_norma[(j)*np.ones(np.size(mean_class,0),dtype=int) ,:]))